### LangSmith Tracing

In [1]:
import os
import getpass

In [2]:
os.environ["LANGSMITH_TRACING"]="true"
os.environ["LANGSMITH_ENDPOINT"]="https://api.smith.langchain.com"
os.environ["LANGSMITH_API_KEY"]= getpass.getpass()
os.environ["LANGSMITH_PROJECT"]="RAG_workflow"

 ···················································


In [1]:
from bs4 import BeautifulSoup
from langchain.schema import Document

In [2]:
file_paths = ["/Users/rajithamuthukrishnan/Desktop/git/datasets/1/html/0009-01.html","/Users/rajithamuthukrishnan/Desktop/git/datasets/1/html/0015-01.html"]

In [3]:
len(file_paths)

2

# Extract Data and convert to LangDocs

In [4]:
def create_lang_documents(raw_docs):
    lang_docs = [Document(page_content=doc['body_text'], metadata={**doc['metadata']})
       for doc in raw_docs]
    return lang_docs

def extract_case_metadata(html_content: str) -> dict:
    soup = BeautifulSoup(html_content, "html.parser")
    
    body_text = soup.get_text(separator=" ", strip=True)
    # Extract - case id
    section = soup.find("section", {"class": "casebody"})
    case_id = section.get("data-case-id") if section else None
    # Extract - case title / name
    h4 = soup.find("h4", {"parties"})
    name = h4.get_text(strip=True) if h4 else None
    # Extract - attorneys
    attorneys = [
        tag.get_text(strip=True) for tag in soup.find_all("p",{"class", "attorneys"})
    ]
    # Extract - author
    author = soup.find("p",{"class","author"}).get_text(strip=True) if soup.find("p",{"class","author"}) else None
    
    return {
        "metadata":{
            "case_id": case_id,
            "case_name": name,
            "attorneys": attorneys,
            "author": author
        },
        "body_text": body_text
    }

def extract_data(file_list):
    data = []
    for file in file_list:
        if file.endswith('.html') or file.endswith('.htm'):
            with open(file, "r", encoding="utf-8") as f:
                html_content = f.read()
                data.append(extract_case_metadata(html_content)) 
                docs = create_lang_documents(data)
        else:
            print(f"Unsupported file type: {file}") 
    return docs

In [5]:
docs = extract_data(file_paths)

In [6]:
# docs

## Chunk docs

In [7]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

In [8]:
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1000,
    chunk_overlap=100,
    add_start_index=True,
)

In [9]:
chunks = text_splitter.split_documents(docs)

In [10]:
# chunks

## Embed the chunks

In [11]:
from langchain_ollama import OllamaEmbeddings
from langchain_community.vectorstores import FAISS

In [12]:
embedding = OllamaEmbeddings(
    model = "all-minilm:l6-v2",
)

# faiss_vectorstore = FAISS.from_documents(chunks, embedding)

In [13]:
# embedded_query=embedding.embed_query("Who is Bennet J?")

In [38]:
# faiss_vectorstore.similarity_search(
#     "Who is Bennet J",
#     k=2
# )

In [40]:
from langchain_experimental.text_splitter import SemanticChunker

In [41]:
semantic_chunker = SemanticChunker(embedding, breakpoint_threshold_type="percentile")
semantic_chunks = semantic_chunker.create_documents([d.page_content for d in chunks])
#
for semantic_chunk in semantic_chunks:
  if "district of Sonoma" in semantic_chunk.page_content:
    print(semantic_chunk.page_content)
    print(len(semantic_chunk.page_content))

comes up on tbe petition of the defendants to be discharged from, the custody of the sheriff of the district of Sonoma, under a writ of habeas corpus heretofore issued by this court. The return of the sheriff shows that the petitioners are detained by him by virtue of an order of the judge of First Instance of the distinct of Sonoma, and that such order was made upon the return of a warrant of arrest against the defendants, charging them with the commission of various felonious acts. Accompanying the return of the sheriff is also to be found a large amount of testimony taken on the examination, going to show that several Indians in the Nappa Valley were shot on the 2Ith day of February last, their lodges burned, and a considerable quantity of wheat, barley, and other property destroyed, and tending to fix tbe perpetration of these acts upon tbe petitioners. It is claimed by the counsel for the accused : 1st.
921
Secondly. The objection to the several orders of commitment is, that it is

In [42]:
semantic_chunk_vectorstore = FAISS.from_documents(semantic_chunks, embedding=embedding)
semantic_chunk_retriever = semantic_chunk_vectorstore.as_retriever(search_kwargs={"k" : 1})

In [43]:
semantic_chunk_retriever.invoke("Which district is mentioned?")

[Document(id='645fbaf6-f2ce-4574-9a9f-774645a04eba', metadata={}, page_content='said district of Sonoma at the “nest first term of said court to be held in and for said district, “ by virtue and in pursuance of the statute laws now passed or “ which may hereafter be passed by the legislature of the state “ now in session.” It is claimed that this authorizes an indefinite imprisonment, inasmuch as the district courts are not yet organized, and the district judges not yet appointed. The order, it is true, might have been drawn up with a greater degree of formality and technical accuracy.')]

# CONDITIONAL RAG

In [15]:
from langchain_ollama import OllamaLLM
from typing import TypedDict, List
from pydantic import BaseModel, Field
from langchain_core.messages import BaseMessage, HumanMessage, AIMessage
from langchain.output_parsers import PydanticOutputParser
from langchain.prompts import ChatPromptTemplate
from langgraph.graph import START, StateGraph
from langgraph.checkpoint.memory import MemorySaver

In [16]:
model = OllamaLLM(
    model = "mistral"
)

### Workflow
1. START - accepts the message/question
2. CLASSIFICATION / TAGGING - tag the text for greetings, small talk or others
3. ROUTER - RAG if Others, LLM if Greetings / Small talk
4. - RAG : Retrieve and Generate
  - LLM : Straight to LLM  
5. Build the graph and add memorysaver

# STEP 1: Classification / Tagging

In [17]:
class State(TypedDict):
    question: str
    topic: str
    context: List[Document]
    answer: str
    messages: List[BaseMessage]

class TopicClassification(BaseModel):
    topic:str = Field(description="Intent of the text - greetings, small talk, law, others")

parser = PydanticOutputParser(pydantic_object=TopicClassification)

def classification(state: State):
    
    question = state["question"]
    categories = ["Greetings", "Small Talk", "Law", "Others"]
    format_instructions = parser.get_format_instructions()
    
    prompt = ChatPromptTemplate.from_messages([
    ("system", "You are a text tagging bot. Classify the user's message into three categories: {categories}.\n\n "
     "Output the result in the required JSON format."
     "\n\n{format_instructions}"),
    ("human", "{question}")
    ])
    chain = (
        prompt.partial(format_instructions=format_instructions)
        | model
        | parser
    )
    response = chain.invoke({"question": question, "categories":", ".join(categories)})
    return {"topic": response.topic}

## STEP 2: Router Logic

In [29]:
def router(state: State):
    print(state['topic'].lower())
    if state["topic"].lower() in ["law","others"]:
        return "RAG"
    else:
        return "LLM"

## STEP 3.1: RAG

In [44]:
# RAG retrieve
def rag_retrieve(state: State):
    print("Entering RAG retrieve")
    # Recursive Text Splitter
#     retrieved_docs = faiss_vectorstore.similarity_search(state["question"], k=2)
    # Semantic chunk retriever
    retrieved_docs = semantic_chunk_retriever.invoke(state["question"], k=2)
    return {'context': retrieved_docs}

In [45]:
# RAG generate
def rag_generate(state: State):
    print("Entering RAG generate")
    #Retrieve messages history and append current user question
    messages = state.get("messages", [])
    messages.append(HumanMessage(content=state["question"]))
    
    question = state["question"]
    context = "\n\n".join(doc.page_content for doc in state["context"]) 
    qa_prompt = ChatPromptTemplate.from_template("""Answer the following question based only on the provided context. 
If you cannot answer please respond with "I don't know:
    <context>
    {context}
    </context>
    Question: {question}""")
    
    
    
    prompt = qa_prompt.invoke({"question": question, "context":context}).to_string()
    # Append prompt to message history
    messages.append(HumanMessage(content=prompt))
    
    # Call LLM with full conversation history including the current question and context
    answer = model.invoke(messages)
    messages.append(AIMessage(content=answer))
    
    return {'answer': answer, 'messages':messages}

## STEP 3.2: LLM

In [46]:
# LLM for small talk
def small_talk(state: State):
    print("Entering small talk")
#     question = state["question"]
    messages = state.get("messages", [])
    messages.append(HumanMessage(content=state["question"]))
#     answer = model.invoke(question)
    answer = model.invoke(messages)
    messages.append(AIMessage(content=answer))
    
    return {"answer": answer, "messages": messages}

## STEP 4: Graph

In [47]:
graph = StateGraph(State)
graph.add_node("classify", classification)
graph.add_node("small_talk_llm", small_talk)
graph.add_node("rag_retrieve", rag_retrieve)
graph.add_node("rag_generate", rag_generate)

graph.add_edge(START, "classify")
graph.add_conditional_edges("classify", router, {
    "LLM": "small_talk_llm",
    "RAG": "rag_retrieve",
})
graph.add_edge("rag_retrieve", "rag_generate")

app = graph.compile(checkpointer=MemorySaver())

In [48]:
config = {"configurable": {"thread_id": "fwegtwg"}}
app.invoke({"question":"Hello my name is bob"}, config)

greetings
Entering small talk


{'question': 'Hello my name is bob',
 'topic': 'Greetings',
 'answer': " Hello Bob! It's nice to meet you. How can I help you today?",
 'messages': [HumanMessage(content='Hello my name is bob', additional_kwargs={}, response_metadata={}),
  AIMessage(content=" Hello Bob! It's nice to meet you. How can I help you today?", additional_kwargs={}, response_metadata={})]}

In [49]:
app.invoke({"question":"Do you know the case titles?"}, config)

others
Entering RAG retrieve
Entering RAG generate


{'question': 'Do you know the case titles?',
 'topic': 'Others',
 'context': [Document(id='6a58b8e1-71a6-4171-b9d5-04fa944b29d2', metadata={}, page_content='the accused : 1st. That the affidavit upon which the warrant of arrest was issued, is defective. 2d.'),
  Document(id='b44bb966-f2e4-4360-a644-354a89949a5a', metadata={}, page_content='Semple and John B. Weller, for tbe applicants, and by G. J. O. Keioen, (attorney general,) for tbe people. By the Court. Bennett, J. This case comes up on tbe petition of the defendants to be discharged from, the custody of the sheriff of')],
 'answer': ' Based on the provided context, it appears that the case title is not explicitly stated in the text. However, a possible deduction for the case title could be "The People vs. [Defendants\' Names]," as it seems to be a criminal case with one party being represented by the Attorney General (for the people). But without more context or an explicit mention of the case title, this is just an educated gues

In [50]:
app.invoke({"question":"What happened in Nappa Valley?"}, config)

others
Entering RAG retrieve
Entering RAG generate


{'question': 'What happened in Nappa Valley?',
 'topic': 'Others',
 'context': [Document(id='cd59bbb1-c35c-4e8f-8dd7-9b0553e486ca', metadata={}, page_content='comes up on tbe petition of the defendants to be discharged from, the custody of the sheriff of the district of Sonoma, under a writ of habeas corpus heretofore issued by this court. The return of the sheriff shows that the petitioners are detained by him by virtue of an order of the judge of First Instance of the distinct of Sonoma, and that such order was made upon the return of a warrant of arrest against the defendants, charging them with the commission of various felonious acts. Accompanying the return of the sheriff is also to be found a large amount of testimony taken on the examination, going to show that several Indians in the Nappa Valley were shot on the 2Ith day of February last, their lodges burned, and a considerable quantity of wheat, barley, and other property destroyed, and tending to fix tbe perpetration of thes

In [51]:
app.invoke({"question":"Who are the defendents?"}, config)

law
Entering RAG retrieve
Entering RAG generate


{'question': 'Who are the defendents?',
 'topic': 'Law',
 'context': [Document(id='1559295d-74d0-4088-8563-0aeba0c3d748', metadata={}, page_content='(Id. 43.) The same power is conferred upon a variety of officers in England and the United States by simply declaring them by statute to be conservators of the peace ; and the constitution of this state confers the same authority in the same terms, upon the justices of this court and the district judges. Being conservators of the peace, the judges of First *14 Instance may'),
  Document(id='18867aab-a150-43ff-b998-f0e0c97693d0', metadata={}, page_content='in positive terms as within the knowledge of the deponent, the commission of the offences charged therein, and to proceed upon information as to the names only of the persons who were guilty of the perpetration of them.')],
 'answer': " Based on the provided context, it's not explicitly stated who the defendants are in this text. The focus is more on the authority of judges as conservator

In [52]:
app.invoke({"question":"What is mentioned about the digest of the mexican law?"}, config)

law
Entering RAG retrieve
Entering RAG generate


{'question': 'What is mentioned about the digest of the mexican law?',
 'topic': 'Law',
 'context': [Document(id='b0ac50dd-fa66-4666-8b3e-bba70c94bfdd', metadata={}, page_content='of sec. 2d, part 2d, Halleck’s *17 Translation anti Digest of Mexican Laws, it is declared, “ that in the trial of causes which exceed $100, but do not exceed $200, the judges shall take cognizance by means of a written process according to law, but without appeal, except the laws have been violated which regulate the mode of proceeding.” From which we infer that the courts of First Instance, so long as they comply with the ordinary rules of practice, and do not violate the laws regulating practice in their courts, have exclusive jurisdiction in all controversies where the matter in dispute shall not exceed $200, and shall exceed $100 ; and no court can review by appeal their judgments in such cases. Entertaining these views, we do not think it'),
  Document(id='202529d6-92f0-4a75-acf3-8220d35bbe1b', metadata

In [53]:
app.invoke({"question":"What is the verdict in the Luther va Master and Owners of Ship Apollo case?"}, config)

law
Entering RAG retrieve
Entering RAG generate


{'question': 'What is the verdict in the Luther va Master and Owners of Ship Apollo case?',
 'topic': 'Law',
 'context': [Document(id='219b435a-7cdd-4ded-87b1-3d6d5f31b7a6', metadata={}, page_content="Luther, respondent, vs. The Master and Owners of Ship Apollo, appellants. This court has no jurisdiction of an appeal from the court of First Instance, where the judgment appealed from is for less than the sum of two hundred dollars; and •where an appeal was brought from a judgment of $166 80, it was ordered that the appeal should be dismissed with costs. The facts are sufficiently stated in the opinion of the court. The cause was argued exjyarte, by Horace Hawes, for the appellants. By the Court, Hastings, Ch. J. This was an action instituted before the court of First Instance for the district of San Francisco, for the recovery of seaman's wages, amounting to the sum of $166 *16 80, and judgment was rendered for the appellee in the sum of $100. The 4th sec."),
  Document(id='bb31c645-b85

In [54]:
app.invoke({"question":"What is the Luther case about?"}, config)

law
Entering RAG retrieve
Entering RAG generate


{'question': 'What is the Luther case about?',
 'topic': 'Law',
 'context': [Document(id='219b435a-7cdd-4ded-87b1-3d6d5f31b7a6', metadata={}, page_content="Luther, respondent, vs. The Master and Owners of Ship Apollo, appellants. This court has no jurisdiction of an appeal from the court of First Instance, where the judgment appealed from is for less than the sum of two hundred dollars; and •where an appeal was brought from a judgment of $166 80, it was ordered that the appeal should be dismissed with costs. The facts are sufficiently stated in the opinion of the court. The cause was argued exjyarte, by Horace Hawes, for the appellants. By the Court, Hastings, Ch. J. This was an action instituted before the court of First Instance for the district of San Francisco, for the recovery of seaman's wages, amounting to the sum of $166 *16 80, and judgment was rendered for the appellee in the sum of $100. The 4th sec."),
  Document(id='bb31c645-b851-4a44-b2d3-cf20ea134761', metadata={}, page_